# シェル

## 通常の実行

In [ ]:
`ls -a`

## 実行方法をカスタマイズ

シェルを実行するためのいくつかの関数があり,ここではそれぞれを見ていく  
- 実行内容を指定する方法によって,次のように区別できる
	* 文字列型  
		`"ls -a"` のようなもの  
		シェルで入力する文字列そのものである  
		スペースなどを含む引数は適宜エスケープが必要
	* 配列型  
		`["ls","-a"]` のようなもの  
		引数をエスケープせずにそのまま渡せる
- `&`を前置するパラメータは,その変数に値が書き込まれる
- `?`を後置するパラメータは,省略可能であることを示す

### `passthru($cmdStr,&$code?)` -> `void`
* stdin,stdout,stderr はインターセプトされない
* 実行完了まで `passthru` より先には進まない
* `$cmdStr` : 実行内容 (文字列型)
* `$code` : 終了コードが変数に書き込まれる

### `exec($cmdStr,&$stdOut?,&$code?)` -> `string`
* stdout がインターセプトされる
* stdin,stderr はインターセプトされない
* 実行完了まで `exec` より先には進まない
* `$cmdStr` : 実行内容 (文字列型)
* `$code` : 終了コードが変数に書き込まれる
* 戻り値 : stdout の最後の行

### `system($cmdStr,&$code?)` -> `string`
* stdin,stdout,stderr はインターセプトされない
* 実行完了まで `system` より先には進まない
* `$cmdStr` : 実行内容 (文字列型)
* `$code` : 終了コードが変数に書き込まれる
* 戻り値 : stdout の最後の行

### `` `$cmdStr` `` -> `string`
### `shell_exec($cmdStr)` -> `string`
* stdout がインターセプトされる
* stdin,stderr はインターセプトされない
* 実行完了まで `shell_exec` より先には進まない
* `$cmdStr` : 実行内容 (文字列型)
* 戻り値 : stdout 全体
* 出力がない場合,エラーで終了した場合は `null` が返される
* シェルを介して実行される (`$cmdStr` の前に `sh -c` などが付加されて実行する)

### `popen($cmdStr,"r")` -> `handler`
* stdout がインターセプトされる
* stdin,stderr はインターセプトされない
* `popen` の時点ではプロセスは終了していないので, `fclose` や `pclose` を用いて終了を待つ必要がある
* `$cmdStr` : 実行内容 (文字列型)
* 戻り値 : 読み込みハンドラ
* 使い方の例 (出力を1024バイト分読み取る)
	```PHP
	$io=popen("コマンド","r");
	$stdOut=fread($io,1024);
	fclose($io);
	```
* 時間をおいて複数回出力されるおそれがある場合,その回数だけ `fclose` の前に `fread` を用意しておかないと全て受け取れない。逆に, `fread` が多すぎると,永遠に受け取りを試み続ける。

### `popen($cmdStr,"w")` -> `handler`
* stdin がインターセプトされる
* stdout,stderr はインターセプトされない
* `popen` の時点ではプロセスは終了していないので, `pclose` を用いて終了を待つ必要がある
* `$cmdStr` : 実行内容 (文字列型)
* 戻り値 : 書き込みハンドラ
* 使い方の例 (出力を1024バイト分読み取る)
	```PHP
	$io=popen("コマンド","w");
	fwrite($io,$stdIn);
	fclose($io);
	```

### `proc_open($cmd,$desc,&$pipes,$cwd?,$env?)` -> `resource`
* stdin,stdout,stderr はインターセプトされない (`$desc`で変更可能)
* `proc_open` の時点ではプロセスは終了していないので, `proc_close` を用いて終了を待つ必要があるΩ
* `$cmd` : 実行内容 (文字列型/配列型)
`$desc` : IOの制御を指定する配列  
	`[stdinの指定,stdoutの指定,stderrの指定]` の形式で指定する  
	指定可能な値は次のとおり
	* `$desc[0]`
		* `["pipe","r"]` … `$pipes` からハンドラで文字列を書き込む
		* `["file","php://stdin","r"]` … 親プロセスのstdinに接続
		* `["file","パス","r"]` … ファイルから読み込む
		* `["file","/dev/null","r"]` … 空の入力
		* ハンドラ … 別のプロセスや `fopen` で開いたファイルハンドラから読み込む (パイプ)
	* `$desc[1]`,`$desc[2]`
		* `["pipe","w"]` … `$pipes` からハンドラで文字列を受け取れる
		* `["file","php://stdout","w"]` … 親プロセスのstdoutに接続
		* `["file","php://stderr","w"]` … 親プロセスのstderrに接続
		* `["file","パス","w"]` … ファイルに書き込む
		* `["file","/dev/null","w"]` … 出力を破棄
		* ハンドラ … 別のプロセスや `fopen` で開いたファイルハンドラに書き込む (パイプ)
* `$pipes` : pipeにしたIOのハンドラの配列が書き込まれる
	* `$pipes[0]` がstdinの書き込みハンドラ, `$pipes[1]`,`$pipes[2]` がstdout,stderr の読み込みハンドラ
	* popenの時と同じように読み込みや書き込むができる
	* `$pipes` のハンドラを別の `proc_open` の `$desc` に渡せば2つのプロセスの間でパイプが作れる
* `$cwd` : カレントディレクトリを指定  
	指定しなければ現在のカレントディレクトリを使用
* `$env` : 環境変数の連想配列  
	指定しなければ現在の環境変数を使用
* 戻り値 : リソース
	実行に失敗すれば `false` が返される
	終了後に必ず `proc_close` をする。`proc_close` から終了コードが得られる。
	`proc_get_status` よりプロセスの情報が得られる
* `proc_get_status(リソース)` ->
	```PHP
	[
		command=>string # 実行コマンド
		pid=>int # プロセスID
		exitcode=>int # 終了コード
		running=>bool # まだ実行中かどうか
	]
	```
* 使い方の例
	```PHP
	$res=proc_open(
		["コマンド","arg1","arg2"],
		[0=>["pipe","w"],1=>STDOUT,2=>["file","/dev/null","w"]],
		$pipes
	);
	if (is_resource($res)) {
		fwrite($pipes[0],$stdIn);
		fclose($pipes[0]);
		$code=proc_close($res);
	}
	```